In [1]:
# 拟合并且保存图片
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.patches import Circle
import os
# from scipy.optimize import curve_fit


def circle(x, cx, cy, r):
    return np.sqrt(r**2 - (x - cx)**2) + cy

# 创建一个列表，用于存储圆心和半径
circle_data = []
error_data = []
# 读取黑白图片
input_folder = './p2-7_extracted_cropped_binary/p5_extracted/'
output_folder = './fitted_extracted/fitted_p5/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# os.path.join(output_path, 'fitted_'+filename)

i=1

for filename in os.listdir(input_folder):
    print(i)
    i=i+1

    # 创建一个新的 Figure 对象
    plt.figure()
    image = cv2.imread(os.path.join(input_folder, filename), cv2.IMREAD_GRAYSCALE)
    # image = cv2.imread('./cropped/cropped_p2-1_0015.bmp', cv2.IMREAD_GRAYSCALE)
    width, length = image.shape
    print(width)
    
    cropped_image = image

    # 获取裁剪后图像的形状
    rows, cols = cropped_image.shape

    # 存储每列的第一个黑色点的坐标
    first_black_pixels = []

    # 遍历每一列
    for col in range(cols):
    # 找到当前列中第一个黑色点的行坐标
        black_pixel_row = np.argmax(cropped_image[:, col] == 0)
        
        # 如果找到了黑色点，则记录坐标
        if black_pixel_row > 0:  # 0 表示未找到黑色点
            # 转换坐标为完整图像坐标
            full_image_row =width - (0 + black_pixel_row)
            first_black_pixels.append((0 + col, full_image_row))

    # 将坐标转换为 NumPy 数组
    first_black_pixels = np.array(first_black_pixels)
    # print(first_black_pixels)

    # 进行圆形拟合
    # p0 = [np.mean(first_black_pixels[:, 0]), np.mean(first_black_pixels[:, 1]), 10.0]  # 初始猜测值
    p0 = [np.mean(first_black_pixels[:, 0]), np.mean(first_black_pixels[:, 1])-200, 200.0]  # 初始猜测值
    params, covariance = curve_fit(circle, first_black_pixels[:, 0], first_black_pixels[:, 1], p0=p0)

    predicted_y = circle(first_black_pixels[:, 0], *params)

    # 计算拟合误差
    y_loss = np.sqrt(np.mean((first_black_pixels[:, 1] - predicted_y)**2))
    print('y_loss:', y_loss)
    r_squared = 1 - (np.sum((first_black_pixels[:, 1] - circle(first_black_pixels[:, 0], *params))**2) / np.sum((first_black_pixels[:, 1] - np.mean(first_black_pixels[:, 1]))**2))
    print('r_squared:', r_squared)


    error_data.append((round(y_loss, 3), round(r_squared, 3)))
#     fit_errors_pix = np.sqrt(np.diag(covariance))
#     # 提取拟合结果
#     cx, cy, r = params

#     # 生成拟合曲线的 x 值范围
#     fit_x = np.linspace(min(first_black_pixels[:, 0]), max(first_black_pixels[:, 0]), 100)
    
#     # 计算拟合曲线的 y 值
#     fit_y = circle(fit_x, cx, cy, r)

#     # 显示图像

#     plt.imshow(image, cmap='gray')

#     # 在坐标轴上画出每列的第一个黑色点的位置
#     plt.scatter(first_black_pixels[:, 0], width-first_black_pixels[:, 1], color='red', s=5)
#     # 画出拟合圆
#     # plt.plot(fit_x, fit_y, color='blue')
#     circle_patch = Circle((cx, width-cy), r, color='purple', fill=False)
#     plt.gca().add_patch(circle_patch)


#     scale_factor = 41 / length  # 将369像素映射到29mm

#     # 标记圆心位置
#     plt.scatter(cx, width-cy, color='green', marker='x')

#     p1= [1.2, 7.8]
#     p1_scaled=[0,0]
#     p1_scaled[0] = (p1[0]+13)/scale_factor
#     p1_scaled[1] = width - (p1[1]+6)/scale_factor 
#     print( "焊接位置/mm：", p1[0]+13, p1[1]+6) 

#     print(p1_scaled)
#     plt.scatter(p1_scaled[0], p1_scaled[1], color='red', marker='x')

#     # # 显示图像、黑色像素位置和拟合圆
#     # plt.axis('equal')
#     # plt.show()
#     output_filename = os.path.join(output_folder, 'fitted_'+filename.replace('bmp','jpg'))
#     plt.savefig(output_filename)
#     print("圆半径",r*scale_factor,"mm")


#     scaled_first_black_pixels = first_black_pixels * scale_factor
#     # p0_scaled = [175 * scale_factor, 60 * scale_factor, 150.0 * scale_factor]  # 初始猜测值
#     p0_scaled = [p * scale_factor for p in p0]
#     params_scaled, covariance_scaled = curve_fit(circle, scaled_first_black_pixels[:, 0], scaled_first_black_pixels[:, 1], p0=p0_scaled)
#     # 计算标准偏差作为拟合误差的估计
#     fit_errors_mm = np.sqrt(np.diag(covariance_scaled))
#     # print(f"Circle center (cx_scaled/mm, cy_scaled/mm): ({ cx*scale_factor -13:.2f} ± {fit_errors[0]:.2f}, {cy*scale_factor - 6:.2f} ± {fit_errors[1]:.2f})")
#     # print(f"Circle radius (r_scaled/mm): {r*scale_factor:.2f} ± {fit_errors[2]:.2f}")

#     print(f"Circle center (cx_scaled/pix, cy_scaled/pix): ({(cx -170):.2f} ± {fit_errors_pix[0]:.2f}, {(580 - cy):.2f} ± {fit_errors_pix[1]:.2f})")
#     print(f"Circle radius (r_scaled/pix): {r:.2f} ± {fit_errors_pix[2]:.2f}")
#     circle_data.append((round(cx , 3), round(580 - cy , 3), round(r, 3)))
#     error_data.append((round(fit_errors_pix[0], 3), round(fit_errors_pix[1], 3), round(fit_errors_pix[2], 3)))



# # 读取文件
# # with open('predict.txt', 'r') as f:
# with open('./Data/labels-240301-p1_p6.txt', 'r') as f:
#     reader = csv.reader(f)
#     data = list(reader)

# # 找到第一个第四、五、六列为空的行的索引
# start_index = next((i for i, row in enumerate(data) if all(x == '' for x in row[3:6])), len(data))

# # 添加新的数据
# # for i in range(start_index, start_index + len(circle_data[:200])):
# #     if i < len(data):
# #         data[i][3:6] = circle_data[i - start_index]
# #     else:
# #         data.append(circle_data[i - start_index] + [''] * (len(data[0]) - 6))

# for i in range (600, 800):
#     data[i][3:6] = circle_data[i - 600] 

# # 写回文件
# # with open('predict.txt', 'w', newline='') as f:
# with open('./Data/labels-240301-p1_p6.txt', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(data)


# cv2.imwrite(output_filename, image)

1
580
y_loss: 2.651856019433685
r_squared: 0.987988970874737
2
580
y_loss: 2.7359853521868027
r_squared: 0.9871214970083659
3
580
y_loss: 2.6535780160182925
r_squared: 0.9876265496053666
4
580
y_loss: 2.3563213064463393
r_squared: 0.989845779100239
5
580
y_loss: 1.7715420719263173
r_squared: 0.9940341284860235
6
580
y_loss: 1.8592794919466
r_squared: 0.9935192795918717
7
580
y_loss: 2.182066090777141
r_squared: 0.9912114253863966
8
580
y_loss: 3.425900236532556
r_squared: 0.9800664343586262
9
580
y_loss: 4.038394505349104
r_squared: 0.9741779417854668
10
580
y_loss: 3.8463509146503223
r_squared: 0.9765527772153605
11
580
y_loss: 3.663952432328529
r_squared: 0.9777344010752975
12
580
y_loss: 3.883151370235016
r_squared: 0.9751257620642423
13
580
y_loss: 3.780315843838245
r_squared: 0.9756972042244244
14
580
y_loss: 3.804682656537501
r_squared: 0.9747820601472434
15
580
y_loss: 3.5752620292219235
r_squared: 0.975753846543058
16
580
y_loss: 4.054866623131489
r_squared: 0.9673344510225211


C:\Users\Kevin\AppData\Local\Temp\ipykernel_36120\1565726814.py:32: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


y_loss: 3.3529196688713383
r_squared: 0.9613431104440601
32
580
y_loss: 5.4342897911628345
r_squared: 0.9253445796759426
33
580
y_loss: 2.8541586256456726
r_squared: 0.962960895878978
34
580
y_loss: 3.046738589089635
r_squared: 0.9562224984925018
35
580
y_loss: 3.0769289103573034
r_squared: 0.9545757120514486
36
580
y_loss: 2.7070859030922727
r_squared: 0.9624617640857042
37
580
y_loss: 2.998504135878304
r_squared: 0.9568666289351947
38
580
y_loss: 2.528380987966411
r_squared: 0.9664575388004323
39
580
y_loss: 2.620405078626142
r_squared: 0.9696440037093058
40
580
y_loss: 3.25386148314991
r_squared: 0.9631405070080151
41
580
y_loss: 2.119711154127062
r_squared: 0.9832954170832868
42
580
y_loss: 1.7470162908443727
r_squared: 0.9888103489688073
43
580
y_loss: 1.2060935981058865
r_squared: 0.9947990199855028
44
580
y_loss: 1.723960828680622
r_squared: 0.9906322948744255
45
580


C:\Users\Kevin\AppData\Local\Temp\ipykernel_36120\1565726814.py:13: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(r**2 - (x - cx)**2) + cy


y_loss: 3.6680842801905715
r_squared: 0.9746349238176437
46
580
y_loss: 2.688508330951267
r_squared: 0.9860249394805566
47
580
y_loss: 2.5321027607286175
r_squared: 0.9873557467357348
48
580
y_loss: 1.5931841676255218
r_squared: 0.994643662029971
49
580
y_loss: 1.7167975584326172
r_squared: 0.9932753386057889
50
580
y_loss: 1.212325023784071
r_squared: 0.9965162205235957
51
580
y_loss: 1.5381659287513156
r_squared: 0.9943442011420413
52
580
y_loss: 1.9143221946098095
r_squared: 0.9904879222631732
53
580
y_loss: 1.895092581610253
r_squared: 0.9899269129074573
54
580
y_loss: 2.018944097469664
r_squared: 0.9880086839733314
55
580
y_loss: 2.3452554446491365
r_squared: 0.9833305326664632
56
580
y_loss: 2.481580678840283
r_squared: 0.9808990133500829
57
580
y_loss: 2.250275470651301
r_squared: 0.9838795801808923
58
580
y_loss: 2.319430640310075
r_squared: 0.9829098739015012
59
580
y_loss: 2.550532511013321
r_squared: 0.9802045870178917
60
580
y_loss: 2.788441566400873
r_squared: 0.9776304032

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [2]:
with open('error.txt', 'r') as f:
    reader = csv.reader(f)
    error_txt = list(reader)
print(len(error_txt))

# for _ in range(1200 - len(error_txt)):
#     error_txt.append([''] * 3)
# print(len(error_txt))

# # 添加新的数据
for i in range(len(error_txt)):
    # 如果原有的列数不足6列，用空字符串填充
    while len(error_txt[i]) < 5:
        error_txt[i].append('')

for i in range (600, 800):
    error_txt[i][0:2] = error_data[i - 600]
    # 写回文件
with open('error.txt', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(error_txt)

1200
